In [ ]:
#@title Connecting to google drive 🥇
from google.colab import drive
drive.mount('/content/drive/')


from google.colab import auth
from google.auth import default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)


# importing libraries
import pandas as pd
import shutil
import os
import datetime 
import numpy as np
#
import re

Mounted at /content/drive/


In [ ]:
#@title Adding Weekly Payout Period serial no. and Run this CELL (Before Proceding Please add payout period and its no.)
week_period={'17 Oct 2022 - 23 Oct 2022':1,'24 Oct 2022 - 30 Oct 2022':2,'31 Oct 2022 - 06 Nov 2022':3,'07 Nov 2022 - 13 Nov 2022':4,
  '14 Nov 2022 - 20 Nov 2022':5,
  '21 Nov 2022 - 27 Nov 2022':6,
  '28 Nov 2022 - 04 Dec 2022':7,
  '05 Dec 2022 - 11 Dec 2022':8,
  '12 Dec 2022 - 18 Dec 2022':9,
  '19 Dec 2022 - 25 Dec 2022':10,
  '26 Dec 2022 - 01 Jan 2023':11,
  '02 Jan 2023 - 08 Jan 2023':12,
  '09 Jan 2023 - 15 Jan 2023':13,
  '16 Jan 2023 - 22 Jan 2023':14,
  '23 Jan 2023 - 29 Jan 2023':15,
  '30 Jan 2023 - 05 Feb 2023':16,
  '06 Feb 2023 - 12 Feb 2023':17,
  '13 Feb 2023 - 19 Feb 2023':18,
  '20 Feb 2023 - 26 Feb 2023':19,
  '27 Feb 2023 - 05 Mar 2023':20,
  '06 Mar 2023 - 12 Mar 2023':21,
  '13 Mar 2023 - 19 Mar 2023':22,
  '20 Mar 2023 - 26 Mar 2023':23,
  '27 Mar 2023 - 02 Apr 2023':24,
  '03 Apr 2023 - 09 Apr 2023':25,
  '10 Apr 2023 - 16 Apr 2023':26,
  '17 Apr 2023 - 23 Apr 2023':27,
  '24 Apr 2023 - 30 Apr 2023':28,
  '01 May 2023 - 07 May 2023':29,
  '08 May 2023 - 14 May 2023':30,
  '15 May 2023 - 21 May 2023':31}

## Long Format i.e. Raw Payout file Format 💊

In [ ]:
#@title Main Code

path='/content/drive/MyDrive/Partners Reports Temporary Save 🧩/Zomato/Zomato Reports'  #paste ur path link here




# ********************************************************** MAIN CODE *************************************************************************

count=0
original=pd.DataFrame()


#for root, dirs, files in os.walk(path):
for file in os.listdir(path):
  get_excel=pd.ExcelFile(f'{path}/'+file)
  count+=1
  payout_breakup=pd.read_excel(get_excel,'Payout breakup',skiprows=range(4))
  payout_breakup.insert(0,'Source_files',file,allow_duplicates=False)

  #df2=pd.payout_breakup(get_excel,'Order level',skiprows=range(6))
  summary=pd.read_excel(get_excel,'Summary')

  payout_breakup.insert(3,'Payout_Report_Period',summary.iloc[2]['Unnamed: 2'],allow_duplicates=False)
  payout_breakup.insert(1,'Res_id',summary.iloc[3]['Unnamed: 2'],allow_duplicates=False)
  payout_breakup.rename(columns={'Particular':'Particular' },inplace=True)

  payout_breakup.insert(2,'Res_Name',summary.iloc[6]['Unnamed: 2'],allow_duplicates=False)
  original=pd.concat([original,payout_breakup])
print(count)



data=original

#@title doroping colms
data.drop(columns=['Unnamed: 0','S.No.'],inplace=True)


#@title droping rows from 'Particular' which have nulls

data.dropna(axis='index',how='all',subset=['Particular' ,'Delivered Orders','Cancelled/ Rejected Orders','Total'],inplace=True)


#@title adding Particulars_alpa_serials
Particulars_alpa_serials= {'Number of Orders':'A1', 
       'Subtotal (items total)':'A2',
       'Packaging charge':'A3', 
       'Delivery charge':'A4',
       'Restaurant discount [Promo]':'A5',
       'Restaurant discount [Salt, Freebie, Pro & other discounts]':'A6',
       'Net bill value \n[(1) + (2) + (3) - (4) - (5)]':'A7',

       'Total GST collected from customers':'B1',
       'GST retained by Zomato - Amount of tax paid by Zomato under section 9(5)':'B2',
       'GST to be paid by Restaurant partner to Govt.':'B3',
       'Gross Sales\n[ (A) + (6) ]':'B4',

       'Commissionable amount   [(B) - (6) -(3)]':'D1', 
       'Commission value ':'D2',
       'Payment mechanism fee':'D3', 
       'Logistics charge':'D4',
       'Taxes on Zomato fees':'D5',
       'Tax collected at source + TCS IGST amount':'D6', 
       'TDS 194O amount':'D7',
       'Other Order level deductions':'D8',
        'Customer Compensation/Recoupment':'D9',
       'Delivery Charges Recovery':'D10',
       'Credit note/(Debit Note) adjustment':'D11',
       'Promo recovery adjustment':'D12',
       'Amount received in cash (on self delivery orders)':'D13',
       'Pay later Deductions & other Res id Level Deductions \n(Refer to Annexure : "Addition/ Deduction Details")':'D14',
       'Total Deductions \n[ (10)+(11)+(12)+(13)+(14)+(15)+(16)+(17)+(18) ]':'D15',

       'Cancellation Refund (Auto)':'E16',
       'TDS 194 H and other Res id level Additions\n(Refer to Annexure : "Addition/ Deduction Details")':'E17',
       'Total Additions \n[(19) + (20)]':'E18',

       'Net Payout \n[(B) - (C) + (D) - (7)]':'F19', 
       'Amount Settled':'F20',
       'Pending Amount (Will be credited on next pay-out day)':'F21'}

#Particulars_alpa_serials
data['serial_no']=data['Particular'].map(Particulars_alpa_serials)


pattern = r'\d{2}\s(\w{3}\s\d{4})\s-\s\d{2}\s\w{3}\s\d{4}'
# extract the month and year from the payout report period column
data['Month']=data['Payout_Report_Period'].str.extract(pattern, expand=False)


data.fillna(0,inplace=True)

#week_no

data['Period_Week_No']=data['Payout_Report_Period'].map(week_period)

## ***********************
os.environ['PYTHONWARNINGS'] = 'always::Warning'
pd.options.mode.chained_assignment = None
os.environ['PYTHONWARNINGS'] = '' 
## ***********************

data.head(2)
# it worked (:

12


,Source_files,Res_id,Res_Name,Payout_Report_Period,Particular,Delivered Orders,Cancelled/ Rejected Orders,Total,serial_no,Month,Period_Week_No
1,Zomato_Settlement_Report_19461401_15 May 2023_...,19461401,Alpha Biryani,15 May 2023 - 21 May 2023,Number of Orders,95.0,1,96.0,A1,May 2023,31
2,Zomato_Settlement_Report_19461401_15 May 2023_...,19461401,Alpha Biryani,15 May 2023 - 21 May 2023,Subtotal (items total),38839.0,0,38839.0,A2,May 2023,31


## Converting to wide format data 📍

In [ ]:
#@title Read data from above long format and then apply Zomato wide format for status "Total"  file - 1

# ws = gc.open_by_key("1lbQe-3w9vdYOu8SbRI79jBWOAs3ny79j9U1Ru10QlEw").sheet1

# rows=ws.get_all_values()
# df=pd.DataFrame.from_records(rows[1:],columns=rows[0])

df=data

df['Res_name_id']=df['Res_Name']+"_"+df['Res_id']


df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

pivoted_total=df.pivot_table(columns=['serial_no','Particular'],index=['Source_files','Res_name_id','Payout_Report_Period','Month','Period_Week_No'],values=['Total'],aggfunc='sum')
pivoted_total['Status']='Total'
df_total=pd.DataFrame(pivoted_total)
df_total=df_total.reset_index()
df_total.columns=df_total.columns.droplevel(1)


df_total.columns=df_total.columns.droplevel()
df_total=df_total.reset_index()
df_total.columns.values[0]='Serial_no'
df_total.columns.values[1]='Source_file'
df_total.columns.values[2]='Res_name_id'
df_total.columns.values[3]='New Payout Period'
df_total.columns.values[4]='Month_Year'
df_total.columns.values[-1]='Status'
df_total.drop(columns=df_total.columns.values[5],inplace=True)


df_total.fillna(0,inplace=True)
df_total.head(1)


Particular,Serial_no,Source_file,Res_name_id,New Payout Period,Month_Year,Extra Inventory Ads (Order Level Deduction),Number of Orders,Subtotal (items total),Packaging charge,Delivery charge,...,TDS 194O amount,Other Order level deductions,Customer Compensation/Recoupment,Cancellation Refund (Auto),"TDS 194 H and other Res id level Additions\n(Refer to Annexure : ""Addition/ Deduction Details"")",Total Additions \n[(19) + (20)],Net Payout \n[(B) - (C) + (D) - (7)],Amount Settled,Pending Amount (Will be credited on next pay-out day),Status
0,0,Zomato_Settlement_Report_18387409_15 May 2023_...,Dilli Darbar_18387409,15 May 2023 - 21 May 2023,May 2023,0.0,646.0,194972.0,13821.0,0.0,...,1756.5,0.0,0.0,168.41,0.0,168.41,106605.05,106603.18,0.0,Total


In [ ]:
#@title Zomato wide format  for status "cancelled"  file - 2
pivoted_cancelled=df.pivot_table(columns=['serial_no','Particular'],index=['Source_files','Res_name_id','Payout_Report_Period','Month','Period_Week_No'],values=['Cancelled/ Rejected Orders'],aggfunc='sum')
pivoted_cancelled['Status']='Cancelled/ Rejected Orders'
df_cancelled=pd.DataFrame(pivoted_cancelled)
df_cancelled=df_cancelled.reset_index()
df_cancelled.columns=df_cancelled.columns.droplevel(1)


df_cancelled.columns=df_cancelled.columns.droplevel()
df_cancelled=df_cancelled.reset_index()
df_cancelled.columns.values[0]='Serial_no'
df_cancelled.columns.values[1]='Source_file'
df_cancelled.columns.values[2]='Res_name_id'
df_cancelled.columns.values[3]='New Payout Period'
df_cancelled.columns.values[4]='Month_Year'
df_cancelled.columns.values[-1]='Status'
df_cancelled.drop(columns=df_cancelled.columns.values[5],inplace=True)
# df_cancelled   = df_cancelled[['Serial_no', 'Source_file', 'Res_name_id', 'New Payout Period',
#                       'Month_Year','Status' ,
#                       'Number of Orders', 'Subtotal (items total)', 'Packaging charge',
#                       'Delivery charge', 'Restaurant discount [Promo]',
#                       'Restaurant discount [Salt, Freebie, Pro & other discounts]',
#                       'Net bill value \n[(1) + (2) + (3) - (4) - (5)]',
#                       'Total GST collected from customers',
#                       'GST retained by Zomato - Amount of tax paid by Zomato under section 9(5)',
#                       'GST to be paid by Restaurant partner to Govt.',
#                       'Gross Sales\n[ (A) + (6) ]',
#                       'Commissionable amount   [(B) - (6) -(3)]', 'Delivery Charges Recovery',
#                       'Credit note/(Debit Note) adjustment', 'Promo recovery adjustment',
#                       'Amount received in cash (on self delivery orders)',
#                       'Pay later Deductions & other Res id Level Deductions \n(Refer to Annexure : "Addition/ Deduction Details")',
#                       'Total Deductions \n[ (10)+(11)+(12)+(13)+(14)+(15)+(16)+(17)+(18) ]',
#                       'Commission value', 'Payment mechanism fee', 'Logistics charge',
#                       'Taxes on Zomato fees', 'Tax collected at source + TCS IGST amount',
#                       'TDS 194O amount', 'Other Order level deductions',
#                       'Customer Compensation/Recoupment',
#                       'Extra Inventory Ads (Order Level Deduction)', 
#                       'Cancellation Refund (Auto)',
#                       'TDS 194 H and other Res id level Additions\n(Refer to Annexure : "Addition/ Deduction Details")',
#                       'Total Additions \n[(19) + (20)]',
#                       'Net Payout \n[(B) - (C) + (D) - (7)]', 'Amount Settled',
#                       'Pending Amount (Will be credited on next pay-out day)']]

df_cancelled.fillna(0,inplace=True)
df_cancelled.head(1)


Particular,Serial_no,Source_file,Res_name_id,New Payout Period,Month_Year,Extra Inventory Ads (Order Level Deduction),Number of Orders,Subtotal (items total),Packaging charge,Delivery charge,...,TDS 194O amount,Other Order level deductions,Customer Compensation/Recoupment,Cancellation Refund (Auto),"TDS 194 H and other Res id level Additions\n(Refer to Annexure : ""Addition/ Deduction Details"")",Total Additions \n[(19) + (20)],Net Payout \n[(B) - (C) + (D) - (7)],Amount Settled,Pending Amount (Will be credited on next pay-out day),Status
0,0,Zomato_Settlement_Report_18387409_15 May 2023_...,Dilli Darbar_18387409,15 May 2023 - 21 May 2023,May 2023,0,1,0,0,0,...,0,0,0,168.41,-,168.41,168.41,168.41,0,Cancelled/ Rejected Orders


In [ ]:
#@title Zomato wide format  for status "delivered"  file - 3
pivoted_delivered=df.pivot_table(columns=['serial_no','Particular'],index=['Source_files','Res_name_id','Payout_Report_Period','Month','Period_Week_No'],values=['Delivered Orders'],aggfunc='sum')
pivoted_delivered['Status']='Delivered Orders'
df_delivered=pd.DataFrame(pivoted_delivered)
df_delivered=df_delivered.reset_index()
df_delivered.columns=df_delivered.columns.droplevel(1)


df_delivered.columns=df_delivered.columns.droplevel()
df_delivered=df_delivered.reset_index()
df_delivered.columns.values[0]='Serial_no'
df_delivered.columns.values[1]='Source_file'
df_delivered.columns.values[2]='Res_name_id'
df_delivered.columns.values[3]='New Payout Period'
df_delivered.columns.values[4]='Month_Year'
df_delivered.columns.values[-1]='Status'
df_delivered.drop(columns=df_delivered.columns.values[5],inplace=True)


df_delivered.fillna(0,inplace=True)
df_delivered.head(1)


Particular,Serial_no,Source_file,Res_name_id,New Payout Period,Month_Year,Extra Inventory Ads (Order Level Deduction),Number of Orders,Subtotal (items total),Packaging charge,Delivery charge,...,TDS 194O amount,Other Order level deductions,Customer Compensation/Recoupment,Cancellation Refund (Auto),"TDS 194 H and other Res id level Additions\n(Refer to Annexure : ""Addition/ Deduction Details"")",Total Additions \n[(19) + (20)],Net Payout \n[(B) - (C) + (D) - (7)],Amount Settled,Pending Amount (Will be credited on next pay-out day),Status
0,0,Zomato_Settlement_Report_18387409_15 May 2023_...,Dilli Darbar_18387409,15 May 2023 - 21 May 2023,May 2023,0.0,645.0,194972.0,13821.0,0.0,...,1756.5,0.0,0.0,0.0,0.0,0.0,106436.64,106434.77,0.0,Delivered Orders


In [ ]:
#@title Merging the data and adding payout period serial no.

final_data=pd.concat([df_total,df_delivered,df_cancelled],ignore_index=True)


week_no=[w_no for w_no in week_period.values()]
#week_no

final_data['Period_Week_No']=final_data['New Payout Period'].map(week_period)



In [ ]:
#@title Pushing the data to sheet

sheetid = '1DI_uXhjKt09UFYfXR0H5Mv98ce3TW6o16iBOWD_iVG4'


# ******************************** Main COde ************************************************************************
import gspread_dataframe as gd

ws = gc.open_by_key(f'{sheetid}').sheet1
existing = gd.get_as_dataframe(ws)
updated = existing.append(final_data)
gd.set_with_dataframe(ws, updated)


<ipython-input-8-ce67d2fd32df>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updated = existing.append(final_data)


## Move files to respective folders

In [ ]:
#@title Zomato Move the files to their respective Res_name folders

source_folder = '/content/drive/MyDrive/Partners Reports Temporary Save 🧩/Zomato/Zomato Reports'  #paste ur path link here
destination_folder = '/content/drive/MyDrive/01_Sales_Reports/C_Zomato/2023'






# *************************************************************** MAIN CODE ****************************************************************************

zom_data=data
zom_data['Payout Month']=zom_data['Payout_Report_Period'].str.split(' ').str[1]


invoice_data = zom_data

# Loop through each row in the invoice data
for index, row in invoice_data.iterrows():

    # Create the destination folder path i.e. Res name folders
    payout_month_folder = os.path.join(destination_folder, row['Payout Month'])
    res_name_folder = os.path.join(payout_month_folder, row['Res_Name'])


    # Create the destination folders if they don't exist
    if not os.path.exists(payout_month_folder):
        os.mkdir(payout_month_folder)

    if not os.path.exists(res_name_folder):
        os.mkdir(res_name_folder)

    # Construct the source file path and destination file path
    source_file_path = os.path.join(source_folder, row['Source_files'])
    destination_file_path = os.path.join(res_name_folder, row['Source_files'])

    if not os.path.exists(source_file_path):
      continue

    shutil.move(source_file_path, destination_file_path)
print('Done!')

Done!


## Monthly DC 🧩

In [ ]:
#@title Main Code

month_path='/content/drive/MyDrive/01_Sales_Reports/C_Zomato/2022/Old_Format'

count2=0
Month_original=pd.DataFrame()

files = glob.glob(os.path.join(month_path , "*.csv"))

for file in files:
  rd_csv=pd.read_csv(file)
  count2+=1
  rd_csv.insert(0,'Source file',os.path.basename(file),allow_duplicates=False)
  Month_original=pd.concat([Month_original,rd_csv])


mon_data=Month_original


mon_data.drop(columns=['S.no.','Mode of payment','Order type','Delivery State','Ice cream deductions - Hyperpure (INR)','Ice cream handling charge (INR)',
'Settlement status','Settlement date', 'Bank UTR', 'Customer ID','Extra Inventory Ads (INR)'],inplace=True)


mon_data['Order_date']=pd.to_datetime(mon_data['Order Date'].str.split(' ',expand=True)[0])


mon_data['Month']=mon_data['Order_date'].dt.month_name().str.slice(stop=3)
mon_data['Year']=mon_data['Order_date'].dt.year


mon_data['Month_Year']=mon_data['Month'].astype(str)+" "+mon_data['Year'].astype(str)

print('Total Files - ',count2)


261


In [ ]:
#@title Pushing the data to sheet
import gspread_dataframe as gd
ws=gc.open_by_key('1qHY2HepZAnVtAaw9l0czjtP_JK5FSifUcQXSHxPCUq8')
wc=ws.worksheet('Month')

existing_sheet=gd.get_as_dataframe(wc)
updated=existing_sheet.append(mon_data)
gd.set_with_dataframe(wc,updated)

In [ ]:
#@title if files not in folder then move files from one folder to another 
# Set the source and destination directories
sor = '/content/drive/MyDrive/Partners Reports Temporary Save 🧩/Swiggy Reports'
des = '/content/drive/MyDrive/01_Sales_Reports/B_Swiggy/2023'

count = 0

# Loop through each file in the source directory
for file in os.listdir(sor):
    src_path = os.path.join(sor, file)
    dest_path = os.path.join(des, file)
    if not os.path.exists(dest_path):      
#      shutil.move(src_path, dest_path)  
      count += 1

# Print the total number of files moved
print(f"{count} files moved")
